<a href="https://colab.research.google.com/github/danielpy108/MachineLearningAlgorithms/blob/master/00_LinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Regression problem

In [0]:
from sklearn.datasets import load_diabetes

In [3]:
training_data = load_diabetes()

X = training_data['data']
y = training_data['target']
features = training_data['feature_names']
description = training_data['DESCR']

print(f'Data samples: {X.shape[0]}')
print(f'Input features (dimenssion): {X.shape[1]}')
print(f"Features: {features}")
print(description)

Data samples: 442
Input features (dimenssion): 10
Features: ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - Age
      - Sex
      - Body mass index
      - Average blood pressure
      - S1
      - S2
      - S3
      - S4
      - S5
      - S6

Note: Each of these 10 feature variables have been mean centered and scaled by the standard deviation times `n_samples` (i.e. the sum of squares of 

In [4]:
# Turn the data into a pandas DataFrame
df = pd.DataFrame(np.concatenate((X, y.reshape(-1, 1)), axis=1), columns=features+['Output'])
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,Output
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


In [0]:
# We need to separate the dataset into training and test samples
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, train_size=0.2)

In [0]:
# Now, the data will be turned into a pytorch tensor for the linear
# regression model
X_training_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_training_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

## Creating the linear model

In [0]:
class LinearRegression():
    def __init__(self, nsamples, nfeatures):
        self.W = torch.randn(nfeatures, requires_grad=True)
        self.b = torch.randn(1, requires_grad=True)
    
    def pred(self, x):
        return x@self.W + self.b
    
    def loss(self, y, y_pred):
        squared_error = (y - y_pred).pow(2)
        N = torch.numel(squared_error)
        return 1/(2*N) * torch.sum(squared_error)
    
    def fit(self, X, y, lr, epochs):
        Loss = []
        for e in range(epochs):
            y_pred = self.pred(X)
            loss = self.loss(y, y_pred)
            Loss.append(loss.item())
            loss.backward()
            with torch.no_grad():
                self.W -=  lr*self.W.grad
                self.b -= lr*self.b.grad
                self.W.grad.zero_()
                self.b.grad.zero_()
            if (e+1)%10 == 0:
                print(f"Epoch [{e}/{epochs}] - Loss: {loss}")
        return np.array(Loss)

In [0]:
torch.manual_seed(1234)

nsamples = X_training_tensor.shape[0]
nfeatures = X_training_tensor.shape[1]

lr = LinearRegression(nsamples, nfeatures)

In [98]:
Loss = lr.fit(X_training_tensor, y_training_tensor, lr=0.1, epochs=1000)

Epoch [9/1000] - Loss: 4083.1005859375
Epoch [19/1000] - Loss: 2639.25
Epoch [29/1000] - Loss: 2445.451904296875
Epoch [39/1000] - Loss: 2403.946533203125
Epoch [49/1000] - Loss: 2381.29052734375
Epoch [59/1000] - Loss: 2361.2607421875
Epoch [69/1000] - Loss: 2341.875244140625
Epoch [79/1000] - Loss: 2322.887939453125
Epoch [89/1000] - Loss: 2304.260986328125
Epoch [99/1000] - Loss: 2285.98583984375
Epoch [109/1000] - Loss: 2268.05322265625
Epoch [119/1000] - Loss: 2250.455810546875
Epoch [129/1000] - Loss: 2233.188232421875
Epoch [139/1000] - Loss: 2216.241943359375
Epoch [149/1000] - Loss: 2199.61083984375
Epoch [159/1000] - Loss: 2183.2890625
Epoch [169/1000] - Loss: 2167.27001953125
Epoch [179/1000] - Loss: 2151.54736328125
Epoch [189/1000] - Loss: 2136.115478515625
Epoch [199/1000] - Loss: 2120.968017578125
Epoch [209/1000] - Loss: 2106.098876953125
Epoch [219/1000] - Loss: 2091.503173828125
Epoch [229/1000] - Loss: 2077.17431640625
Epoch [239/1000] - Loss: 2063.108642578125
Epoch

In [99]:
# Plotting the loss

figure = go.Figure()
figure.add_trace(
    go.Scatter(
        x=np.arange(Loss.size),
        y=Loss,
        mode='lines+markers',
        name='Training loss',
        line=go.scatter.Line(color='blue')
    )
)
figure.update_layout(
    title='Training loss',
    title_font_size=22
)
figure.show()

# Using built in LR model

Pytorch provides built in models in the class _torch.nn_. In order to use it we must define:

* Dataset object within a DataLoader object (optional but convenient)
* Loss function
* Optimizer
* Training loop



In [0]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

In [63]:
# TensorDataset
# Dataset wrapping tensors as a tuple
# Each sample will be retrieved by indexing tensors along the first dimension
# Source: https://pytorch.org/docs/stable/data.html#torch.utils.data.TensorDataset

training_dataset = TensorDataset(X_training_tensor, y_training_tensor)
print(f'First sample:\n{training_dataset[0]}\n')
print(f'First sample (Input tensor):\n{training_dataset[0][0]}\n')
print(f'First sample (Output tensor):\n{training_dataset[0][1]}\n')
print(f'Training samples: {len(training_dataset)}')

First sample:
(tensor([-0.0237, -0.0446,  0.0455,  0.0907, -0.0181, -0.0354,  0.0707, -0.0395,
        -0.0345, -0.0094]), tensor(175.))

First sample (Input tensor):
tensor([-0.0237, -0.0446,  0.0455,  0.0907, -0.0181, -0.0354,  0.0707, -0.0395,
        -0.0345, -0.0094])

First sample (Output tensor):
175.0

Training samples: 88


In [64]:
# Define the DataLoader
# Return an iterable object
batch_size = 5
training_dataloader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
for xi, yi in training_dataloader:
    print(xi)
    print(yi)
    break

# As we can see, in one iteration it fetches 5 data samples

tensor([[-0.0600,  0.0507, -0.0472, -0.0229, -0.0717, -0.0577, -0.0066, -0.0395,
         -0.0629, -0.0549],
        [ 0.0272, -0.0446, -0.0073, -0.0504,  0.0755,  0.0566,  0.0339, -0.0026,
          0.0434,  0.0155],
        [ 0.0054, -0.0446,  0.0585, -0.0435, -0.0731, -0.0724,  0.0192, -0.0764,
         -0.0514, -0.0259],
        [ 0.0054,  0.0507, -0.0084,  0.0219,  0.0548,  0.0732, -0.0250,  0.0343,
          0.0126,  0.0942],
        [ 0.0018,  0.0507, -0.0062, -0.0194, -0.0098,  0.0049, -0.0397,  0.0343,
          0.0148,  0.0983]])
tensor([ 72.,  95., 136., 100., 262.])


In [65]:
# Creating the model
model = nn.Linear(len(features), 1)

print(model.weight, end='\n\n')
print(model.bias)

Parameter containing:
tensor([[-0.2748,  0.3052,  0.3019, -0.2177,  0.0782,  0.2614,  0.0612,  0.1360,
          0.2355,  0.1342]], requires_grad=True)

Parameter containing:
tensor([-0.0572], requires_grad=True)


In [66]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.2748,  0.3052,  0.3019, -0.2177,  0.0782,  0.2614,  0.0612,  0.1360,
           0.2355,  0.1342]], requires_grad=True), Parameter containing:
 tensor([-0.0572], requires_grad=True)]

In [112]:
# Loss function
# We will use the MSE as we did in the manually created model
loss_fn = F.mse_loss

# Optimizer 
# Stochastic Gradient Descent
# Using mini batches in each iteration instead of one by one
opt = torch.optim.SGD(model.parameters(), lr=1e-05)
opt

SGD (
Parameter Group 0
    dampening: 0
    lr: 1e-05
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [0]:
# Defining the training loop
def fit(epochs, model, loss_fn, optimizer):
    Loss = []
    for e in range(epochs):
        for xi, yi in training_dataloader:
            y_pred = model(xi)
            loss = loss_fn(y_pred, yi)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        if (e+1)%10 == 0:
            print(f'Epoch[{e}/{epochs}], Loss = {loss}')
        Loss.append(loss.item())
    return np.array(Loss)

In [116]:
model_Loss = fit(50, model=model, loss_fn=loss_fn, optimizer=opt)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning:

Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning:

Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch[9/50], Loss = 3216.864990234375
Epoch[19/50], Loss = 3730.44482421875
Epoch[29/50], Loss = 2586.797119140625
Epoch[39/50], Loss = 3359.46728515625
Epoch[49/50], Loss = 1582.2579345703125


In [117]:
figure = go.Figure()
figure.add_trace(
    go.Scatter(
        x=np.arange(model_Loss.size),
        y=model_Loss,
        mode='lines+markers',
        name='Training loss',
        line=go.scatter.Line(color='blue')
    )
)
figure.update_layout(
    title='Training loss',
    title_font_size=22
)
figure.show()